# TP2 Robotica Movel - Potential Fields

## Alunos
- Bernardo Nogueira Borges 2020006396
- Daniele Cassia Silva Diniz 2020076874


### Importando Libs

In [376]:
from coppeliasim_zmqremoteapi_client import RemoteAPIClient
import numpy as np

client = RemoteAPIClient()
sim = client.require('sim')
np.set_printoptions(precision=3,suppress=True)

### Definindo Parametros

In [377]:

# Star
q_init = np.array([-8,-8])
q_goal = np.array([-2,8])

## Funções Potencial

### Força de Atração
$$F_{att}(q) = k_{att} \cdot (q_{goal} - q)$$

In [378]:
def ForceAttraction(q, q_goal, katt=0.2):
    return katt * (q_goal - q)

### Força de repulsão
$$F_{rep,i}(q) = -\nabla U_{rep,i}(q)$$
$$=\begin{cases}
  \begin{aligned}
    \frac{k_{rep,i}}{\rho_i^2(q)}\left( \frac{1}{\rho_i(q)} - \frac{1}{\rho_{0,i}} \right)^{\gamma-1} \frac{q - q_{obs,i}}{\rho_i(q)} &, \textnormal{se } \rho_i(q) \le \rho_{0,i}\\
    0&, \textnormal{se } \rho_i(q) \gt \rho_{0,i}\\
  \end{aligned}
\end{cases}
$$

In [379]:
LIMIT_RANGE = 2 

def ForceRepulsion(q, q_obs_i, krep=.5):
    d = np.linalg.norm(q - q_obs_i)

    if d > LIMIT_RANGE:
        return np.array([0.0,0.0])
    
    return (krep / d**2) * ((1/d) - (1/LIMIT_RANGE)) * (q-q_obs_i) / d


### Força Resultante 
$$F(q)  = F_{att}(q) + \sum_{i=1}^{p}{F_{rep,i}(q)}$$

In [380]:
def ForceResult(q,q_goal,q_obs):
    ans = ForceAttraction(q,q_goal) 

    for i in range(len(q_obs)):
        ans += ForceRepulsion(q,q_obs[i])

    # Random, para evitar minimos locais
    # ans += np.random_uniform(-0.04,0.04,size=2)
    
    return ans

### Controladores
- [De Luca e Oriolo, 1994](https://www.researchgate.net/publication/225543929_Control_of_Wheeled_Mobile_Robots_An_Experimental_Overview)
$$v = k_v(\dot x \cos \theta + \dot y \sin \theta)$$
$$\omega = k_{\omega}(atan2(\dot y,\dot x) - \theta)$$
$$u = \begin{bmatrix}
   v \\
   \omega 
\end{bmatrix}
$$

In [381]:
def deLucaOriolo(force,theta):
    kv = 1
    kw = .7
    Fx, Fy = force

    v = kv*(Fx*np.cos(theta) + Fy*np.sin(theta))
    w = kw*(np.arctan2(Fy,Fx) - theta)

    return [v,w]

### Model Cinemático - Robô Diferencial
$$\omega_R = \frac{2v+\omega L}{2r}$$
$$\omega_L = \frac{2v-\omega L}{2r}$$

In [382]:
L = 0.381   # Metros
r = 0.0975  # Metros

def differentialModel(v,w):
    wr = ((2.0*v) + (w*L))/(2.0*r)
    wl = ((2.0*v) - (w*L))/(2.0*r)
    return [wl,wr]
 

## Interação

### Definindo a Leitura do Lazer

In [383]:
def readSensorData():
    string_range_data = sim.getStringSignal("hokuyo_range_data")
    string_angle_data = sim.getStringSignal("hokuyo_angle_data")

    # Verifica se os dados foram obtidos corretamente
    if string_range_data == None or string_angle_data == None: return None
    
    # unpack dos dados de range e angulos do sensor
    raw_range_data = sim.unpackFloatTable(string_range_data)
    raw_angle_data = sim.unpackFloatTable(string_angle_data)
    return raw_range_data, raw_angle_data

def getLaserData():
    # Prosseguindo com as leituras
    # Geralmente a primeira leitura é inválida (atenção ao Operation Mode)
    # Em loop até garantir que as leituras serão válidas
    sensor_data = readSensorData()
    while sensor_data == None:
        sensor_data = readSensorData()

    raw_range_data, raw_angle_data = sensor_data
    laser_data = np.array([raw_angle_data, raw_range_data]).T
    return laser_data


### Iniciando o Robô

In [384]:
sim.stopSimulation()

# Iniciar o Pioneer
pioneer = sim.getObject("/Pioneer")
sim.setObjectPosition(pioneer,[int(q_init[0]),int(q_init[1]),.2])

right_motor = sim.getObject("/Pioneer/rightMotor")
left_motor = sim.getObject("/Pioneer/leftMotor")

sim.setJointTargetVelocity(right_motor, 0)
sim.setJointTargetVelocity(left_motor, 0)

# Set Goal
goal_object = sim.getObject("/Goal")
sim.setObjectPosition(goal_object,[int(q_goal[0]),int(q_goal[1]),.2])

def getObjectXY(obj):
    *q, _ = sim.getObjectPosition(obj)
    return np.array(q[:2])

def getObjectAngle(obj):
    ori = sim.getObjectOrientation(obj)
    return ori[2]

q = getObjectXY(pioneer)

### Transformando Coordenadas

In [385]:

def matrix_rel_robot(name):
    obj = sim.getObject(name)
    M = sim.getObjectMatrix(obj,pioneer)
    M = np.array(M).reshape(-1,4)
    return M

def getObstaclePositions(laser_data):

    # Matriz de transformação laser -> robô
    Trl = matrix_rel_robot("/Pioneer/fastHokuyo")
    Trl = np.vstack([Trl,[0,0,0,1]])

    # Matriz de transformação robô -> mundo
    Twr = sim.getObjectMatrix(pioneer)
    Twr = np.vstack([np.array(Twr).reshape(3,4),[0,0,0,1]])

    def laser_read_to_point(laser_read):
        ang, dist = laser_read
        x = dist * np.cos(ang)
        y = dist * np.sin(ang)
                
        # Ponto de leitura no referencial do laser
        Pl = np.array([x,y,0,1])
        # ponto laser -> robô
        Pr =  Trl @ Pl
        # ponto robô -> mundo
        Pw =  Twr @ Pr
        return np.array([Pw[0],Pw[1]])

    return list(map(laser_read_to_point,laser_data))


### Movimentando o Robô

In [386]:

sim.startSimulation()

while (t := sim.getSimulationTime()) < 300:
    q = getObjectXY(pioneer)
    q_goal = getObjectXY(goal_object)

    error = np.linalg.norm(q_goal - q)
    if error < 0.5:
        print("Goal reached :)")
        break

    laser_data = getLaserData()

    # Filtrando leituras distantes
    laser_data = laser_data[laser_data[:, 1] > LIMIT_RANGE]
    q_obs = getObstaclePositions(laser_data)

    force = ForceResult(q,q_goal,q_obs) 

    theta = getObjectAngle(pioneer)

    # Controle Cinematico
    [v,w] = deLucaOriolo(force,theta)

    # Cinematica inversa
    [wl,wr] = differentialModel(v,w)
 
    # Enviando velocidades
    sim.setJointTargetVelocity(left_motor,wl)
    sim.setJointTargetVelocity(right_motor,wr)

    sim.step()


# Stop
sim.setJointTargetVelocity(left_motor,0)
sim.setJointTargetVelocity(right_motor,0)
sim.stopSimulation()


Goal reached :)
